# Исследование надежности заемщиков


**Задача исследования:**

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

Данные содержатся в файле /datasets/data.csv

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Обзор данных

Импорт необходимых библиотек

In [2]:
import pandas as pd

Выгрузка данных из файла `data.csv` в переменную `data`

In [3]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Вывод первых 10 строчек датафрейма `data` на экран:

In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывод основной информации о датафрейме:

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Вывод количества пропущенных значений для каждого столбца.

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения встречаются в двух столбцах: `days_employed` и `total_income`. 

Столбец `total_income` хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце можно медианным значением по каждому типу из столбца `income_type`. 

Пропуски в столбце `days_employed` будут обработаны на следующем этапе.

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Проверим результат заполнения пропусков в столбце `total_income`

In [8]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Обработка аномальных значений

В столбце `days_employed`(рабочий стаж) встречаются отрицательные значения. Скорее всего это произошло в результате какой-то ошибки, т.к. трудовой стаж не может быть отрицательным. 

Обработаем значения в этом столбце, заменив все отрицательные значения положительными:

In [9]:
data['days_employed'] = data['days_employed'].abs()

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Вывод медианного значение трудового стажа в днях для каждого типа занятости:

In [10]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получились аномально большие значения. Данные столбцы не понадобятся для дальнейшего исследования, оставим данные без изменений.

Вывод перечня уникальных значений столбца `children`:

In [11]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения: "-1" и "20". Проверим сколько раз встречаются аномальные значения в данных:

In [12]:
data.groupby('children')['family_status_id'].agg('count')


children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: family_status_id, dtype: int64

Количество столбцов с аномальными значениями составляют менее 1%, Удаленние данных строк не окажет существенного влияния на результаты исследования. 

In [13]:
data = data[(data['children'] != -1) & (data['children'] != 20)] # удаление строк с аномальными значениями

data.groupby('children')['family_status_id'].agg('count') # проверка результата удаления аномалий

children
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: family_status_id, dtype: int64

**Обработка пропусков в столбце `days_employed`**

Заполним пропуски медианными значениями для каждого типа занятости `income_type`



In [14]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Вывод пропущенных значений для проверки результатов их обработки: 

In [15]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Замена вещественного типа данных в столбце `total_income` на целочисленный с помощью метода `astype()`:

In [16]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

В столбце `education` встречаются одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру:

In [17]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных:

In [18]:
data.duplicated().sum()

71

Количество дубликатов составляет менее одного процента и не оказывает существенного влияния на результаты исследования. Удалим дубликаты:

In [19]:
data = data.drop_duplicates()

data.duplicated().sum() # проверка удаления дубликатов

0

### Категоризация данных

Присвоение категорий в зависимости от уровня дохода: 

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Создание функций для расчета и присвоения категорий:

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

Применение функции к таблице для присвоения категорий

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


Вывод уникальных целей взятия кредита из столбца `purpose`:

In [22]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создание функцит, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

Применение функции к таблице для присвоения категорий

In [24]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


В рамках предобработки обнаружены три проблемы в данных:

* нарушения в стиле заголовков,
* пропущенные значения,
* присутствуют аномальные значения
* дубликаты — явные и неявные.

Для устранения проблем было выполнено следующее:

* испралены заголовки, чтобы упростить работу с таблицей,
пропущенные значения заменены на 'unknown',
* удалены явные и обработаны неявные дубликаты.
* изменен тип данных в столбце `total_income` на целочисленный
* пропуски заполнены медивнными значениями
* обработаны аномальные значения
* добавлены новые столбцы для разбиения клиентов по категориям в зависимости от уровня дохода и цели кредита.

### Исследовательский аналих данных

#### Анализ взаимосвязи между количеством детей и возвратом кредита в срок.


In [25]:
# группировка количества кредитов и количества просрочек по количеству детей
table_children_research = (data.groupby('children')['debt']
                           .agg(credits = ('count'), credits_debt = ('sum'))
                           .sort_values(by='credits', ascending=False).reset_index())

# вычисление доли просрочек от общего числа кредитов
table_children_research['credits_share'] = round(table_children_research['credits_debt']/table_children_research['credits']*100, 2)

# вывод итоговой таблицы
table_children_research

,children,credits,credits_debt,credits_share
0,0,14091,1063,7.54
1,1,4808,444,9.23
2,2,2052,194,9.45
3,3,330,27,8.18
4,4,41,4,9.76
5,5,9,0,0.00


**Вывод:** 

У клиентов с 5 детьми просрочки по платежам отсутствуют, 
но также на них приходится достаточно маленькое число выданных кредитов, чтобы отследить закономерность. Выборка не сбалансирована, классы в выборке имеют разные размеры.

Из анализа оставшихся значений можно сделать вывод, что клиенты без детей имеют меньше просрочек по кредитам, чем клиенты с детьми. Можно сделать вывод, что клиенты без детей являются более ответственными, чем клиенты, у которых есть дети. Видимо, наличие ребенка накладывает на клиента дополнительные финансовое бремя (обязательства) и клиенты либо менее ответственно относятся к своим кредитным обязательствам, либо не имеют возможности в срок оплачивать обязательства. Однако доля просрочек не сильно изменяется с изменением количества детей и нет сильной зависимости между количеством детей и возвратом кредитов в срок. 

Также необходимо учесть факт, что количество детей только зафиксировано на момент выдачи кредита, никто не гарантирует, что их не станет больше в течение срока выплаты. Есть вероятность, что кредитный портфель бездетных (со сроками выданных кредитов по несколько лет) скорее всего со временем ухудшится по показателю возврата денег (какая-то часть бездетных заведут детей, и у них должников из 7,5% станет 9%). И тогда этот риск можно было бы закладывать заранее (но для этого хорошо было бы понимать, сколько бездетных перейдут в категорию «с детьми» в течение 5 лет, например). 

#### Анализ взаимосвязи между семейным положением и возвратом кредита в срок.

In [27]:
# группировка количества кредитов и количества просрочек по семейному положению
research_family_status = (data.groupby('family_status')['debt']
                           .agg(credits = ('count'), credits_debt = ('sum'))
                           .sort_values(by='credits', ascending=False).reset_index())

# вычисление доли просрочек от общего числа кредитов
research_family_status['credits_share'] = round(research_family_status['credits_debt']/research_family_status['credits']*100, 2)

# вывод итоговой таблицы
research_family_status

,family_status,credits,credits_debt,credits_share
0,женат / замужем,12261,927,7.56
1,гражданский брак,4134,385,9.31
2,Не женат / не замужем,2796,273,9.76
3,в разводе,1189,84,7.06
4,вдовец / вдова,951,63,6.62


**Вывод:** 

На основание полученных данных можно сделать вывод, что существует зависимость между семейным положением и возвратом кредита в срок. Люди с семейным положением "вдовец/вдова" имеют меньше просрочек по срокам выплаты кредитов. Получается, что узаконенные отношения (включая и бывшие отношения) ведут к более ответственному отношению к своим обязательствам. 

Больше всего просрочек у людей с семейным положением: "не женат/не замужем". Кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке.

#### Анализ взаимосвязи между уровнем дохода и возвратом кредита в срок.

In [28]:
# группировка количества кредитов и количества просрочек по уровню дохода
income_research = (data.groupby('total_income_category')['debt']
                           .agg(credits = ('count'), credits_debt = ('sum'))
                           .sort_values(by='credits', ascending=False).reset_index())

# вычисление доли просрочек от общего числа кредитов
income_research['credits_share'] = round(income_research['credits_debt']/income_research['credits']*100, 2)

# вывод итоговой таблицы
income_research

,total_income_category,credits,credits_debt,credits_share
0,C,15921,1353,8.50
1,B,5014,354,7.06
2,D,349,21,6.02
3,A,25,2,8.00
4,E,22,2,9.09


**Вывод:** 

На основании полученных данных можно сделать выводы, что есть зависимость между уровнем дохода и возврата кредитов в срок. Меньше всего просрочек по кредитам у клиентов с уровнем дохода от 30001–50000, больше всего у клиентов с уровнем дохода 0–30000.

Ожидаемо, для группы с низким доходом (Е) процент невозвратов самый высокий. Что более интересно, так это то, что с ростом доходов тенденция (D->C и B->А) к снижению числа долгов не сохраняется, а скорее даже растёт. Можно сказать, что по какой-то причине люди "перестают считать деньги" или планировать "наперёд". Из-за подобного поведения наших групп параметр ("доход") нельзя назвать надёжным при определении платёжеспособности клиента. Без дополнительной информации он может даже наоборот привести к выдаче кредита состоятельному, но не слишком "надёжному" клиенту, хотя, возможно, это может быть аномалия, вызванная недостаточным размером выборки для клиентов с категорией доходов А.    
    
Выборка по заемщикам, как с наименьшим доходом, так и с наибольшим крайне недостаточна для формирования однозначных выводов. Можно думать, что E-категория самая низкооплачиваемая не может корректно распоряжаться деньгами и «влезает» в кредиты, но тогда неясно почему у А-категории также достаточно высокий процент задолженности. В общем вывод по группам А и Е неочевиден, слишком мало данных.

Если же взять для анализа только группы В, С и D, то по полученным данным не наблюдаем зависимости "те, кто получает больше ЗП, лучше возвращает кредит». В целом это логично, так как материальный достаток - не единственная переменная, которая влияет на качество кредитной истории.

Также на результат могли оказать влияние 2174 пропуска в данных столбца "total_income". Сводная таблица (или группировка) с данными о зависимости между уровнем дохода и возвратом кредита в срок возможно могла бы выглядеть иначе, если бы не пропущенные данные, которые мы выше заполнили медианным значением зарплаты, исходя из типа занятости соответствующего заемщика.

#### Анализ взаимосвязи между целью кредита и возвратом кредита в срок.

In [29]:
# группировка количества кредитов и количества просрочек по целям кредитов
purpose_research = (data.groupby('purpose_category')['debt']
                           .agg(credits = ('count'), credits_debt = ('sum'))
                           .sort_values(by='credits', ascending=False).reset_index())

# вычисление доли просрочек от общего числа кредитов
purpose_research['credits_share'] = round(purpose_research['credits_debt']/purpose_research['credits']*100, 2)

# вывод итоговой таблицы
purpose_research

,purpose_category,credits,credits_debt,credits_share
0,операции с недвижимостью,10751,780,7.26
1,операции с автомобилем,4279,400,9.35
2,получение образования,3988,369,9.25
3,проведение свадьбы,2313,183,7.91


**Вывод:** 

Клиенты, взявшие кредит по операции с недвижимостью и проведение свадьбы имеют меньше просрочек по срокам возврата, больше просрочек имеют клиенты, взявшие кредит под операции с автомобилем и получение образования. 

# Общий вывод.

В рамках проекта было проведено исследование с целью выявить, какие факторы влияют на факт погашения кредита в срок. 
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает
способность потенциального заёмщика вернуть кредит банку.

Перед проведением исследования были поставлены следующие гипотезы:
* Гипотеза 1: Количество детей влияет на факт погашения кредита в срок.
* Гипотеза 2: Семейное положение влияет на факт погашения кредита в срок.
* Гипотеза 3: Уровень дохода влияет на факт погашения кредита с срок.
* Гипотеза 4: Цель кредита влияет на факт погашения кредита в срок.

**Гипотеза 1 подтверждена частично**

* Доля просроченных кредитов для детей без детей - 7,5%
* Доля просроченных кредитов для детей с 1 ребенком - 9,2% 
* Доля просроченных кредитов для детей с 2 детьми - 9,5%

Для семей с 3,4,5 детьми требуется больше данных для более точных результатов

Рекомендация: провести дополнительное исследование на большей выборке данных.

**Гипотеза 2 подтверждена**

* Доля просроченных кредитов у клиентов со статусом "вдова/вдовец" - 6,6%
* Доля просроченных кредитов у клиентов со статусом "в разводе" - 7,1%
* Доля просроченных кредитов у клиентов со статусом "женат/замужем" - 7,5%
* Доля просроченных кредитов у клиентов со статусом "гражданский брак" - 9,2%
* Доля просроченных кредитов у клиентов со статусом "не женат/не замужем" - 9,7%

Выводы: для клиентов со статусами: "вдова/вдовец", "в разводе", "женат/замужем" меньше вероятность просрочки выплаты по кредитам, для клиентов со статусами "гражданский брак", "не женат/не замужем" данная вероятность выше.

Рекомендация: разница между вероятностями 3,2%, поэтому не рекомендуется наделять данную закономерность высоким весом при принятии решения о кредите.

**Гипотеза 3 частично подтверждена**

* доля просроченных кредитов для заемщиков с уровнем дохода от 50001 до 200000 - 8,5% 
* доля просроченных кредитов для заемщиков с уровнем дохолда от 200001 до 1000000 - 7,0%

Для более точных выводов нужны дополнительные данные по клиентам с уровнем дохода от 0 до 30000, от 30001 до 50000, от 10000001 рублей.

Рекомендация: провести дополнительное исследование на большей выборке данных.

**Гипотеза 4 подтверждена**

* доля просроченных кредитов, целью которых были операции с недвижимостью - 7,3%
* доля просроченных кредитов, целью которых было проведение свадьбы - 7,9%
* доля просроченных кредитов, целью которых было получение образования - 9,3%
* доля просроченных кредитов, целью которых были операции с автомобилем - 9,3%

Рекомендация: разница между минимальной и максимальной вероятностями 2%, поэтому не рекомендуется наделять данную закономерность высоким весом при принятии решения о кредите.

**Общие выводы:**

* Специалисты, которые занимаются построением модели кредитного скоринга, могут использовать зависимости из второй (семейной положение) и четвертой (цели кредита) гипотез
* Необходимо собрать полную выборку для повторной проверки первой (количество детей) и третей (уровень дохода) гипотез






    



